In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [11]:
# CONSTANTS
tubL, tubW, tubH = 1.20, 0.60, 0.40 # meters
molWaterInM3 = 55493.9 # mol / meter^3
humanCP = 62.97 # J / mol K

In [35]:
# VARIABLES
countL, countW, countH = 24, 12, 8 # dimensionless
waterStartingTemp = 37.0 # deg celsius
ambientTemp = 20.0 # deg celsius
bathWallThermalDiffusivity = 22e-6 # cast iron thermal diffusivity
bathWallThermalConductivity = 55 # cast iron W/m K
bathWallSpecificHeat = 500 # J / kg K
bathWallThickness = 5e-3 # cast iron thickness, m
bathWallDensity = 7170 # cast iron density, kg/m^3
kMetal = 0.20

dt = 0.1 # seconds

In [36]:
# DERIVED
distL, distW, distH = tubL/countL, tubW/countW, tubH/countH
saL, saW, saH = distW*distH, distL*distH, distL*distW
wallWtL = saL * (bathWallThickness*bathWallDensity) # kg
wallWtW = saW * (bathWallThickness*bathWallDensity) # kg
wallWtH = saH * (bathWallThickness*bathWallDensity) # kg
#kMetalL = bathWallThermalConductivity / ((bathWallThickness ** 2) * bathWallSpecificHeat * bathWallDensity) # SA * TC / TH * [J/K]
#print(saL, kMetalL)

In [41]:
def getWaterDiffusivity(temp):
    lmda = 0.561 + 0.002 * temp + 0.00000962 * (temp ** 2)
    cp = 75.98 - 0.061 * temp + 0.002 * (temp ** 2) - 0.00001755 * (temp ** 3)
    rho = 999.86 + 0.058 * temp - 0.008 * (temp ** 2) + 0.0000397 * (temp ** 3)
    return lmda / (cp * rho)

def getSubsectionVolumeMols():
    return molWaterInM3 * (tubL/countL) * (tubW/countW) * (tubH/countH)

def getNewTempArray():
    return waterStartingTemp*np.ones((countL+2, countW+2, countH+1))

def getTubSafe(tub, l, w, h):
    l = 0        if l < 0        else l
    l = countL+1 if l > countL+1 else l
    w = 0        if w < 0        else w
    w = countW+1 if w > countW+1 else w
    h = 0        if h < 0        else h
    h = countH+0 if h > countH+0 else h
    return tub[l][w][h]

def getPointDiffusivity(tub, l, w, h):
    if l == 0 or l == countL+1: return bathWallThermalDiffusivity
    if w == 0 or w == countW+1: return bathWallThermalDiffusivity
    if           h == countH+0: return bathWallThermalDiffusivity # h = 0 layer is water
    return getWaterDiffusivity(tub[l][w][h])

def getCellLagrangian(tub, l, w, h):
    ddL = float(getTubSafe(tub, l+1, w, h) + getTubSafe(tub, l-1, w, h) - 2 * getTubSafe(tub, l, w, h)) / (distL ** 2)
    ddW = float(getTubSafe(tub, l, w+1, h) + getTubSafe(tub, l, w-1, h) - 2 * getTubSafe(tub, l, w, h)) / (distW ** 2)
    ddH = float(getTubSafe(tub, l, w, h+1) + getTubSafe(tub, l, w, h-1) - 2 * getTubSafe(tub, l, w, h)) / (distH ** 2)
    return ddL + ddW + ddH

def cellStep(tub):
    diff = np.copy(tub)
    newTub = np.copy(tub)
    for l in range(0, countL+2):
        for w in range(0, countW+2):
            for h in range(0, countH+1):
                wd = getPointDiffusivity(tub, l, w, h)
                dd = getCellLagrangian(tub, l, w, h)
                mm = getSubsectionVolumeMols()
                diff[l][w][h] = dd#dt*mm*wd*dd
                newTub[l][w][h] += dt*mm*wd*dd
                if l == 0 or w == 0 or l == countL+1 or w == countW+1 or h == countH:
                    # h == 0 is water and is excluded from metal decay
                    newTub[l][w][h] += -0.20*dt*(newTub[l][w][h]-ambientTemp)
    return newTub

In [47]:
arr = getNewTempArray();
#arr[1][1][1] = 90

for i in range(0, 10):
    arr = cellStep(arr)
print(arr)

[[[ 33.89974229  33.89974213  33.89973138  33.89929358  33.89081234]
  [ 34.04102362  34.04102138  34.04087618  34.03477364  33.89929358]
  [ 34.0474041   34.04740177  34.04725025  34.04087618  33.89973138]
  [ 34.04755573  34.0475534   34.04740175  34.04102136  33.89974213]
  [ 34.04755573  34.0475534   34.04740175  34.04102136  33.89974213]
  [ 34.0474041   34.04740177  34.04725025  34.04087618  33.89973138]
  [ 34.04102362  34.04102138  34.04087618  34.03477364  33.89929358]
  [ 33.89974229  33.89974213  33.89973138  33.89929358  33.89081234]]

 [[ 34.04102362  34.04102138  34.04087618  34.03477364  33.89929358]
  [ 36.87020526  36.8701983   36.86938811  36.80765017  34.03477364]
  [ 36.93347275  36.93346564  36.93263607  36.86938811  34.04087618]
  [ 36.93430253  36.93429541  36.9334656   36.87019826  34.04102136]
  [ 36.93430253  36.93429541  36.9334656   36.87019826  34.04102136]
  [ 36.93347275  36.93346564  36.93263607  36.86938811  34.04087618]
  [ 36.87020526  36.8701983   36

In [ ]:
print(saL, saW, saH)
print(wallWtL, wallWtW, wallWtH)